In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
os.chdir('/content/drive/My Drive/CD_Kaziranga_ISINE')

In [3]:
pip install gdal

In [14]:
 pip install -U scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=32b0720ada93665bba6686e42d7bd2e797633461039a143abcb76cc0b4074712
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [15]:
import numpy as np
import sys
import imageio
from osgeo import gdal
import time
from Methodology.util.cluster_util import otsu
from Methodology.util.data_prepro import stad_img


In [16]:
def CVA(img_X, img_Y, stad=False):
    # CVA has not affinity transformation consistency, so it is necessary to normalize multi-temporal images to
    # eliminate the radiometric inconsistency between them
    if stad:
        img_X = stad_img(img_X)
        img_Y = stad_img(img_Y)
    img_diff = img_X - img_Y
    L2_norm = np.sqrt(np.sum(np.square(img_diff), axis=0))
    return L2_norm

In [ ]:
def main():
    data_set_X = gdal.Open(r'D:\KNP_2013_2021_CD\try_two\KNP2013_all6B.tif')  # data set X
    data_set_Y = gdal.Open(r'D:\KNP_2013_2021_CD\try_two\KNP2021_all6B.tif')  # data set Y

    img_width = data_set_X.RasterXSize  # image width
    img_height = data_set_X.RasterYSize  # image height

    img_X = data_set_X.ReadAsArray(0, 0, img_width, img_height)
    img_Y = data_set_Y.ReadAsArray(0, 0, img_width, img_height)

    channel, img_height, img_width = img_X.shape
    tic = time.time()
    L2_norm = CVA(img_X, img_Y)

    bcm = np.ones((img_height, img_width))
    thre = otsu(L2_norm.reshape(1, -1))
    bcm[L2_norm > thre] = 255
    bcm = np.reshape(bcm, (img_height, img_width))
    imageio.imwrite(r'D:\KNP_2013_2021_CD\cd_try_two\CVA_kaziranga.png', bcm)
    toc = time.time()
    print(toc - tic)


if __name__ == '__main__':
    main()